In [1]:
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
sy.logger.remove()

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)
parties = [p1,p2]
t1 = sy.Tensor(np.array([1,2,3])).send(p1)
t2 = sy.Tensor(np.array([1,2,3])).send(p2)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!


kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/azureuser


In [3]:
#Creation of training data.
#Should modify to a concatenate type for pointer abstraction.

X = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])
y = np.array([[0,1,1,0]]).T
x_min_vals = np.array([[0,0,0]])
x_max_vals = np.array([[1,1,1]])
y_min_vals = np.array([0])
y_max_vals = np.array([1])

entity = Entity(name="rasswanth")
min_vals = np.array([0]*X.shape[0])
max_vals = np.array([1]*X.shape[0])

X = sy.Tensor(X)
X.public_shape = X.shape
X = X.private(
    x_min_vals,x_max_vals , entities=[entity.name]*X.shape[0],ndept=True,
).tag("trade_flow")

d1 = X.send(p1)

X = MPCTensor(secret=d1,parties=parties,shape=X.shape)

y = sy.Tensor(y)
y.public_shape = y.shape
y = y.private(
    y_min_vals, y_max_vals, entities=[entity.name]*y.shape[0],ndept=True,
).tag("trade_flow")

d2 = y.send(p1)

y = MPCTensor(secret=d2,parties=parties,shape=y.shape)



In [4]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [5]:

syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1

In [ ]:
for j in range(1):
    l1 = relu(X @ syn0)  ; l1.block
    l2 = relu(l1 @ syn1) ; l2.block 
    l2_delta = (y - l2)* relu(l2,deriv=True) ; l2_delta.block
    l1_delta = (l2_delta@(syn1.T)) * relu(l1,deriv=True) ; l1_delta.block
    syn1  = syn1 + l1.T @ l2_delta ; syn1.block
    syn0 =  syn0 + X.T @ l1_delta ; syn0.block

In [6]:
l1 = relu(X @ syn0)  ; l1.block

Computing...:   0%|          | 0/64 [00:00<?, ?it/s]/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in multiply
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 4)
	 .child[0] = <TensorPointer -> canada:eeeb503ca633e0ec9b3162fb7bab4c4b>
	 .child[1] = <TensorPointer -> italy:eeeb503ca633e0ec9b3162fb7bab4c4b>

In [7]:
l2 = relu(l1 @ syn1) ; l2.block 

/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in matmul
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 1)
	 .child[0] = <TensorPointer -> canada:a4369566d34fddbe9eb9ab447061bb85>
	 .child[1] = <TensorPointer -> italy:a4369566d34fddbe9eb9ab447061bb85>

In [8]:
l2_delta = (y - l2)* relu(l2,deriv=True) ; l2_delta.block

Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 1)
	 .child[0] = <TensorPointer -> canada:b7d501bf26ff7cc1581fce1478a9ba5b>
	 .child[1] = <TensorPointer -> italy:b7d501bf26ff7cc1581fce1478a9ba5b>

In [11]:
l1_delta = (l2_delta@(syn1.T)) * relu(l1,deriv=True) ; l1_delta.block

Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.65it/s]




MPCTensor.shape=(4, 4)
	 .child[0] = <TensorPointer -> canada:c2a70790ad2c8968289539821840e5a3>
	 .child[1] = <TensorPointer -> italy:c2a70790ad2c8968289539821840e5a3>

In [ ]:
syn1  = l1.T @ l2_delta  + syn1; syn1.block

In [ ]:
syn0 =  X.T @ l1_delta  + syn0; syn0.block

In [26]:
res = X.T @ l1_delta

In [33]:
v1 = X.T
v2 = l1_delta

In [34]:
def get_val(val):
    t1 = val.child[0].get_copy()
    t2 = val.child[0].get_copy()
    return t1+t2

In [35]:
v1 = get_val(v1)
v2 = get_val(v2)

In [51]:
v1.child.min_vals.to_numpy() @ v2.child.min_vals.to_numpy()

array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])

In [50]:
v2.child.min_vals

<lazyrepeatarray data: [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]] -> shape: (4, 4)>

In [ ]:
syn0_dp = syn0.publish(sigma=1e4)
syn1_dp = syn1.publish(sigma=1e4)

In [ ]:
print(syn0_dp.get_copy())
print(syn1_dp.get_copy())